In [ ]:
%reload_ext autoreload
%autoreload 2

from parse import load_dfs, get_site_dfs

import polars as pl
from polars import col

(
    range_joinyears,
    df_users,
    df_users_registered_cum,
    df_posts_all,
    df_comments_all,
    df_activity_all,
) = load_dfs("../infodump")

SITE = "all"

df_months, df_posts, df_comments, df_activity = get_site_dfs(
    SITE, df_posts_all, df_comments_all, df_activity_all
)

In [ ]:
df_users_monthly = (
    df_months.join(
        df_activity.select("userid", "month"),
        on="month",
        how="left",
        coalesce=True,
    )
    .unique(["userid", "month"])
    .sort("month")
)

In [ ]:
df_users_monthly_unique = df_users_monthly.group_by("month").agg(
    col("userid").n_unique()
)
df_users_monthly_unique

In [ ]:
df_users_monthly_by_joined = (
    df_users_monthly.join(
        df_users.select("userid", "joinyear"),
        on="userid",
        how="left",
        coalesce=True,
    )
    .group_by("month")
    .agg(
        (
            col("joinyear").filter(joinyear=year).count().alias(str(year))
            for year in range_joinyears
        ),
    )
    .drop("month")
)

df_users_monthly_by_joined

In [ ]:
# need to keep js AGE_LABELS in sync with this
bins = [
    0,
    365,
    365 * 5,
    365 * 10,
    365 * 15,
    365 * 100,
]

df_activity_by_age = (
    df_months.join(
        df_activity.select("userid", "datestamp", "month"),
        on="month",
        how="left",
        coalesce=True,
    )
    .join(
        df_users.select("userid", "joindate"),
        on="userid",
        how="left",
        coalesce=True,
    )
    .with_columns(age=(col("datestamp") - col("joindate")).dt.total_days())
    .with_columns(
        col("age").is_between(bins[i], bins[i + 1], closed="left").alias(str(i))
        for i in range(len(bins) - 1)
    )
    .group_by("month")
    .agg(col(str(i)).sum() for i in range(len(bins) - 1))
    .drop("month")
)

df_activity_by_age

In [ ]:
df_first_active = (
    df_months.join(
        df_activity.select("userid", "month"),
        on="month",
        how="left",
        coalesce=True,
    )
    .unique("userid", keep="first")
    .sort("month")
    .group_by("month")
    .agg(col("userid"))
)

df_first_active

In [ ]:
df_users_new = df_months.join(
    df_first_active, on="month", how="left", coalesce=True
).select("month", new=col("userid").list.len())

df_users_new

In [ ]:
df_users_cum = df_users_new.select("month", cum=col("new").cum_sum())

df_users_cum

In [ ]:
df_posts_deleted = (df_months.join(
        df_posts.select("month", "deleted"),
        on="month",
        how="left",
        coalesce=True,
    )
    .group_by("month")
    .agg(
        col("deleted").filter(col("deleted").is_in([1, 3])).len()
    )  # 1: deleted, 3: deleted and closed on Metatalk
)

df_posts_deleted

In [ ]:
topN = [0.01, 0.05, 0.1]

df_activity_by_top_users = (
    df_months.join(
        df_activity.select("month", "userid"), on="month", how="left", coalesce=True
    )
    .group_by("month")
    .agg(
        pl.len(),
        col("userid").unique_counts().alias("counts").sort(descending=True),
    )
    .select(
        (
            col("counts").list.head(col("counts").list.len() * n).list.sum()
            / col("len")
        ).alias(str(n))
        for n in topN
    )
    .with_columns(
        (col(str(topN[i])) - col(str(topN[i - 1]))) for i in range(1, len(topN))
    )
    .select(pl.all().round(3))
)

df_activity_by_top_users